<a href="https://colab.research.google.com/github/BrunoMog/PROJETOS-IF687/blob/main/mlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Início da implementação


In [1]:
#importanto biblioteca
import numpy as np

In [25]:
#setando parâmetros da rede mlp

input_size = 2
hidden_size = 1
output_size = 1

#cada camada intermediária pode ter o mesmo número de unidades computacionais para fins de simplificação ou apenas terem valores diferentes, nessa implementação vamos adotar a segunda abordagem
#para cada camada intermediária - hidden layer, terá um valor relacionado a quantidade de unidades computacionais

hidden_layer_size = [1]

# serão implementadas três funções de ativação para as camadas intermediárias a relu, sigmoid e linear pela qual será indicada, e para a camdada de saída conforme o indicado sigmoid ou linear
# mas a relu também é uma opção
# a primeira função será para camadas intermediárias e a segunda para a camada de saída

activation_functions = ['relu', 'relu']

# o número máximo de épocas que o modelo será treinado
# o objetivo é que não precise chegar ao número máximo de épocas, quando o modelo apresentar overfitting o treinamento já acabe

MAX_EPOCH = 1

# taxa de aprendizado que será utilizada

learning_rate = 0.03

# tamanho dos mini-batchs que será utilizado dos dados de treinamento e teste

batch_size_train = 100

batch_size_test = 20

In [3]:
# setando o gerador de números aleatórios para setar os pesos iniciais

weigths_seed = 115
rng = np.random.default_rng(weigths_seed)

# como a rede mlp é totalmente conectada, o número de pesos gerados entre camadas segirá sempre o padrão layer[i]*layer[i+1]

weights = []

for i in range(hidden_size+1):

  if i == 0:

    weights.append(rng.random(size = (input_size, hidden_layer_size[i], 1)))

  elif i < hidden_size:

    weights.append(rng.random((hidden_layer_size[i-1], hidden_layer_size[i], 1)))

  else:

    weights.append(rng.random((hidden_layer_size[i-1], output_size, 1)))



In [22]:
# primeiro index é para referir a qual camada, sempre haverá hidden_layers+1 camada de pesos
# o segundo index para referenciar qual unidade computacional da camada
# o terceiro index para referenciar o peso a qual unidade computacional da próxima camada
# o quarto index para obter o valor

print(weights[0][0][0][0])

0.7023380241887802


### Funções para o treinamento da rede MLP

#### fase foward e backward


In [23]:
# funções de ativação

# função relu normal
def function_Relu(x):

  return max(0, x)

# função sigmoid normal
def function_Sigmoid(x):

  return 1/(1+np.exp(-x))

# função linear entre 0 e 1
def function_Linear(x):

  return max(0, min(1, x))

In [ ]:
# funções de custo

def mean_square_loss(labels, predictions):

  loss = 0

  for l, p in zip(labels, predictions):
      loss = loss + (l - p) ** 2

  loss = loss / len(labels)

  return loss

In [26]:
# processo de foward da rede

def foward(input_size, hidden_size, output_size, hidden_layer_size, activation_functions, weights, data):

  # o resultado de cada valor multiplicado pelo seu peso será armazenado no valor parcial

  resultado_parcial = [[[] for j in range(hidden_layer_size[i])] for i in range(hidden_size)]

  resultado_parcial.append([[]for i in range(output_size)])

  # calcular o resultado camada por camada

  for i in range(hidden_size+1):

    # no primeiro caso será a camada de dados

    if i == 0:

      input = data

    # o caso em que estamos calculando o resuldado das camadas intermediárias

    if(i < hidden_size):

      for j in range(len(input)):

        for k in range(hidden_layer_size[i]):

          resultado_parcial[i][k].append(weights[i][j][k][0]*input[j])


      aux = np.zeros(hidden_layer_size[i])

      for j in range(hidden_layer_size[i]):

        aux[j] = np.sum(resultado_parcial[i][j])

      # após ter os resultados parciais da soma ponderada das entradas daquela camada, esse valor irá passar pela função de ativação

      if activation_functions[0] == 'relu':

        for j in range(len(aux)):

          aux[j] = function_Relu(aux[j])

      elif activation_functions[0] == 'sigmoid':

        for j in range(len(aux)):

          aux[j] = function_Sigmoid(aux[j])

      elif activation_functions[0] == 'linear':

        for j in range(len(aux)):

          aux[j] = function_Linear(aux[j])


    # para o caso contrário

    else:

      for j in range(len(input)):

        for k in range(output_size):

          resultado_parcial[i][k].append(weights[i][j][k][0]*input[j])

      aux = np.zeros(output_size)

      for j in range(output_size):

        aux[j] = np.sum(resultado_parcial[i][j])

      # após ter os resultados parciais da soma ponderada das entradas daquela camada, esse valor irá passar pela função de ativação

      if activation_functions[0] == 'relu':

        for j in range(len(aux)):

          aux[j] = function_Relu(aux[j])

      elif activation_functions[0] == 'sigmoid':

        for j in range(len(aux)):

          aux[j] = function_Sigmoid(aux[j])

      elif activation_functions[0] == 'linear':

        for j in range(len(aux)):

          aux[j] = function_Linear(aux[j])

    # após ter o resultado parcial da camada, o input da próxima iteração será o resultado dessa

    input = aux

  return input


In [27]:
# o treinamento fará no máximo MAX_EPOCH iterações
# ele irá parar quando a acurácia dos testes diminuir em 5 épocas seguidas
# após o processo de treinamento será retornado os pesos ótimos para o problema encontrado

def treinamento(input_size, hidden_size, output_size, hidden_layer_size, activation_functions, MAX_EPOCH, learning_rate, batch_size_train, batch_size_test, weights, data_train, data_test, data_train_label, data_test_label):

  for epoch in range(MAX_EPOCH):

    # atualizando os batchs de treino e de teste

    batch_index = rng.integers(len(data_train), size = batch_size_train)

    batch_train = data_train[batch_index]

    batch_train_label = data_train_label[batch_index]

    batch_index = rng.integers(len(data_test), size = batch_size_test)

    batch_test = data_test[batch_index]

    batch_test_label = data_test_label[batch_index]

    # atualizando os pesos de acordo com o erro no batch do treino
    # a função de custo será a MSE para regressão, e cross entropy para problemas de classificação

    for i in range(batch_size_train):

      #resultado do processo de foward

      foward_result = foward(input_size, hidden_size, output_size, hidden_layer_size, activation_functions, weights, batch_train[i])

      # atualizando os pesos


  return weights

In [30]:
treinamento(input_size, hidden_size, output_size, hidden_layer_size, activation_functions, MAX_EPOCH, learning_rate, data, data, weights, data, data, data, data)

[0.71071042]


[array([[[0.70233802]],
 
        [[0.60370907]]]),
 array([[[0.54416906]]])]